<a href="https://colab.research.google.com/github/yeipi-mora/python_codes/blob/main/sandpiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Libraries
import numpy as np

In [3]:
'''
  Several functions to deal with topplings and stabilization of
  grid graphs.
'''

def topple_grid(S: np.array, A: np.array, T: np.array, x: int, y: int):
    '''
     Fire a single site in S once
     i.e. increment odometer
     remove chips from S[x, y]
     and give chips to neighbors
     (x+1, y), (x-1, y), (x, y+1), (x, y-1)

     Returns True if we have toppled and False other case
    '''
    # Number of times fire site (x, y)
    # We take max in case there is a hole at (x, y)
    z = max(np.floor(S[x, y] / 4), 0)

    # Increment the odometer
    T[x, y] += z
    S[x, y] -= 4 * z

    # Give sand to each neighbor
    if x > 0:
        S[x - 1, y] += z
    if x < S.shape[0] - 1:
        S[x + 1, y] += z
    if y > 0:
        S[x, y - 1] += z
    if y < S.shape[1] - 1:
        S[x, y + 1] += z

    return z > 0

def stabilize_grid(S: np.array):
    # Initialize the all 0 odometer
    T = np.zeros_like(S)
    # Iterate over all sites and try to topple
    # until it is not possible anymore
    is_topple = True
    while is_topple:
        is_topple = False
        for y in range(S.shape[1]):
            for x in range(S.shape[0]):
                is_topple = is_topple or topple_grid(S, T, x, y)
    return T

def random_config_grid(N: int):
  # Maximmum value of grains we place in one grid
  high = 1000*N
  # Create a random configuration S of size N x N
  S = np.random.randint(low=0, high=high, size=(N, N))

  while True:
      # Pick a random vertex and add one grain
      random_vertex = (np.random.randint(0, high), np.random.randint(0, high))
      S[random_vertex] += 1

      T = stabilize_grid(S)

In [77]:
'''
  Several functions to deal with topplings and stabilization of
  general graphs.
'''

def topple(A: np.ndarray, c: list, v: int):

  '''
    We will topple a graph G represented by adjacency matrix
    Inputs:
      - A: configuration of grains
      - c: adjacency matrix
      - v: vertex that we will topple

      Return:
        - We return a True statement if we topple v. False in other case.
  '''

  # Number of neighbors of v.
  n = np.sum(A[v])
  grains = c[v]

  if c[v] >= n:
    for w in range(len(A)):
      if c[v] >= n and A[v][w] != 0:
        c[w] += 1
        c[v] -= 1
      elif c[v] < n: break
  return grains >= n

def stabilize(A: np.ndarray, c: list):
  '''
    We stabilize a graph G represented by its adjacency matrix A.

    Inputs:
      - A: configuration of grains
      - c: adjacency matrix
    Output:
      - We return the stable configuration
  '''
  # Iterate over all sites and try to topple
  # until it is not possible anymore
  is_topple = True
  while is_topple:
      is_topple = False
      for v in range(len(A)):
          is_topple = is_topple or topple(A,c,v)
  return c

def evolutive(A: dict, c: list):
  '''
    We stabilize over a discrete evolution of graphs
    Inputs:
      - A: a dictionary to stores each graph G_{t} (represented by its adjacency matrix A_{t}).
      A key of A corresponds to the discrete time t.
      - c: an initial configuration.
  '''
  for t in A.keys():
    stabilize(A[t],c)

In [76]:
graph = [
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [0, 1, 1, 0]
]

c = [0, 3, 2, 0]
stabilize(np.array(graph),c)

[1, 2, 2, 0]